Protect Nails

In [0]:

%sql

WITH nail_procedures AS (
  -- Get all tibial nail implant procedures
  SELECT DISTINCT
    id.ENCNTR_ID,
    id.PERSON_ID,
    id.CLINSIG_UPDT_DT_TM as surgery_date,
    id.IMPLANT_DESCRIPTION,
    id.MANUFACTURER,
    id.SIDE_OF_PROCEDURE,
    -- Check if Gentamicin coated (protected type has "protect" in description)
    CASE 
      WHEN UPPER(id.IMPLANT_DESCRIPTION) LIKE '%PROTECT%'
      THEN 'Y'
      ELSE 'N'
    END as is_gentamicin_coated
  FROM 4_prod.bronze.implant_details id
  WHERE UPPER(id.IMPLANT_DESCRIPTION) LIKE '%NAIL%'  
    AND UPPER(id.IMPLANT_DESCRIPTION) LIKE '%TIBIA%' 
    AND (UPPER(id.MANUFACTURER) LIKE '%SYNTHES%' 
         OR UPPER(id.MANUFACTURER) LIKE '%DUPUY%'
         OR UPPER(id.MANUFACTURER) LIKE '%DEPUY%')
),

-- Get laterality from procedure codes
laterality_info AS (
  SELECT 
    mp.ENCNTR_ID,
    CASE 
      WHEN STRING_AGG(CASE WHEN mp.SOURCE_STRING LIKE '%Left%' THEN 'Left' END, ',') IS NOT NULL 
        AND STRING_AGG(CASE WHEN mp.SOURCE_STRING LIKE '%Right%' THEN 'Right' END, ',') IS NOT NULL 
      THEN 'Bilateral'
      WHEN STRING_AGG(CASE WHEN mp.SOURCE_STRING LIKE '%Left%' THEN 'Left' END, ',') IS NOT NULL 
      THEN 'Left'
      WHEN STRING_AGG(CASE WHEN mp.SOURCE_STRING LIKE '%Right%' THEN 'Right' END, ',') IS NOT NULL 
      THEN 'Right'
      ELSE NULL
    END as laterality
  FROM 4_prod.bronze.map_procedure mp
  GROUP BY mp.ENCNTR_ID
),

-- Aggregate procedure codes
procedure_codes AS (
  SELECT 
    mp.ENCNTR_ID,
    STRING_AGG(
      DISTINCT CONCAT(
        COALESCE(mp.SOURCE_IDENTIFIER, ''), 
        CASE 
          WHEN mp.SOURCE_IDENTIFIER IS NOT NULL AND mp.SOURCE_STRING IS NOT NULL THEN '|'
          ELSE ''
        END,
        COALESCE(mp.SOURCE_STRING, '')
      ), 
      '; '
    ) as procedure_list
  FROM 4_prod.bronze.map_procedure mp
  WHERE mp.SOURCE_IDENTIFIER IS NOT NULL OR mp.SOURCE_STRING IS NOT NULL
  GROUP BY mp.ENCNTR_ID
),

-- Aggregate diagnosis codes
diagnosis_codes AS (
  SELECT 
    md.ENCNTR_ID,
    STRING_AGG(
      DISTINCT CONCAT(
        COALESCE(md.SOURCE_IDENTIFIER, ''), 
        CASE 
          WHEN md.SOURCE_IDENTIFIER IS NOT NULL AND md.SOURCE_STRING IS NOT NULL THEN '|'
          ELSE ''
        END,
        COALESCE(md.SOURCE_STRING, '')
      ), 
      '; '
    ) as diagnosis_list
  FROM 4_prod.bronze.map_diagnosis md
  WHERE md.SOURCE_IDENTIFIER IS NOT NULL OR md.SOURCE_STRING IS NOT NULL
  GROUP BY md.ENCNTR_ID
),

-- Check for diabetes
diabetes_check AS (
  SELECT DISTINCT person_id,
    'Y' as has_diabetes
  FROM (
    -- Check problems table for diabetes
    SELECT person_id 
    FROM 4_prod.rde.rde_all_problems 
    WHERE UPPER(code_text) LIKE '%DIABETES MELLITUS%'
    
    UNION
    
    -- Check diagnosis table for diabetes (excluding family history and pregnancy)
    SELECT person_id 
    FROM 4_prod.rde.rde_all_diagnosis 
    WHERE UPPER(code_text) LIKE '%DIABETES MELLITUS%' 
      AND UPPER(code_text) NOT LIKE '%FAMILY HISTORY%'
      AND UPPER(code_text) NOT LIKE '%PREGNANCY%'
  ) diabetes_records
),

-- Check for smoking status
smoker_check AS (
  SELECT DISTINCT person_id,
    'Y' as is_smoker
  FROM 4_prod.rde.rde_all_problems 
  WHERE code_text = 'Current smoker' 
     OR code_text = 'smoker'
),

-- Get most recent height measurements
height_data AS (
  SELECT 
    PERSON_ID,
    TRY_CAST(EventResult AS DOUBLE) AS height_cm,
    ClinicalSignificanceDate AS height_date,
    ROW_NUMBER() OVER (PARTITION BY PERSON_ID ORDER BY ClinicalSignificanceDate DESC) AS rn
  FROM 4_prod.rde.rde_measurements
  WHERE (EventType = 'Height/Length Measured' OR EventType = 'Height (cm)')
    AND UnitsDesc = 'centimetre'
    AND EventResult IS NOT NULL
),

-- Get most recent weight measurements
weight_data AS (
  SELECT 
    PERSON_ID,
    TRY_CAST(EventResult AS DOUBLE) AS weight_kg,
    ClinicalSignificanceDate AS weight_date,
    ROW_NUMBER() OVER (PARTITION BY PERSON_ID ORDER BY ClinicalSignificanceDate DESC) AS rn
  FROM 4_prod.rde.rde_measurements
  WHERE (EventType = 'Weight Measured' OR EventType = 'Weight Estimated')
    AND UnitsDesc = 'Kilogram'
    AND EventResult IS NOT NULL
),

-- Calculate BMI and obesity status
bmi_calc AS (
  SELECT 
    COALESCE(h.PERSON_ID, w.PERSON_ID) AS PERSON_ID,
    h.height_cm,
    w.weight_kg,
    CASE 
      WHEN h.height_cm IS NOT NULL AND w.weight_kg IS NOT NULL AND h.height_cm > 0
      THEN ROUND(w.weight_kg / POWER(h.height_cm / 100.0, 2), 2)
      ELSE NULL
    END AS BMI,
    CASE 
      WHEN h.height_cm IS NOT NULL AND w.weight_kg IS NOT NULL AND h.height_cm > 0
        AND (w.weight_kg / POWER(h.height_cm / 100.0, 2)) >= 30
      THEN 'Y'
      WHEN h.height_cm IS NOT NULL AND w.weight_kg IS NOT NULL AND h.height_cm > 0
        AND (w.weight_kg / POWER(h.height_cm / 100.0, 2)) < 30
      THEN 'N'
      ELSE NULL
    END AS is_obese
  FROM (SELECT * FROM height_data WHERE rn = 1) h
  FULL OUTER JOIN (SELECT * FROM weight_data WHERE rn = 1) w 
    ON h.PERSON_ID = w.PERSON_ID
),

-- Get admission Hemoglobin (first in encounter)
admission_hb AS (
  SELECT 
    ENCNTR_ID,
    admission_hb,
    hb_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      CASE 
        WHEN p.resultUnit = 'g/L' THEN TRY_CAST(p.Result_nbr AS DOUBLE) / 10
        ELSE TRY_CAST(p.Result_nbr AS DOUBLE)
      END AS admission_hb,
      p.RequestDate as hb_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE (p.Testdesc = 'Haemoglobin' 
        OR p.Testdesc = 'Mean Cell Haemoglobin' 
        OR p.Testdesc = 'Mean Cell Haemoglobin Concentration')
      AND (p.resultUnit = 'Grams/decilitre' OR p.resultUnit = 'g/L')
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get admission eGFR (first in encounter)
admission_egfr AS (
  SELECT 
    ENCNTR_ID,
    admission_egfr,
    egfr_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      TRY_CAST(p.Result_nbr AS DOUBLE) AS admission_egfr,
      p.RequestDate as egfr_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE p.Testdesc = 'estimated GFR'
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get admission Creatinine (first in encounter)
admission_creat AS (
  SELECT 
    ENCNTR_ID,
    admission_creatinine,
    creat_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      TRY_CAST(p.Result_nbr AS DOUBLE) AS admission_creatinine,
      p.RequestDate as creat_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE p.Testdesc = 'Creatinine Serum'
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get admission Urea (first in encounter)
admission_urea AS (
  SELECT 
    ENCNTR_ID,
    admission_urea,
    urea_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      TRY_CAST(p.Result_nbr AS DOUBLE) AS admission_urea,
      p.RequestDate as urea_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE p.Testcode = 'UREA'
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get admission Sodium (first in encounter)
admission_sodium AS (
  SELECT 
    ENCNTR_ID,
    admission_sodium,
    sodium_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      TRY_CAST(p.Result_nbr AS DOUBLE) AS admission_sodium,
      p.RequestDate as sodium_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE p.Testdesc = 'Sodium Serum'
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get admission Potassium (first in encounter)
admission_potassium AS (
  SELECT 
    ENCNTR_ID,
    admission_potassium,
    potassium_date
  FROM (
    SELECT 
      np.ENCNTR_ID,
      TRY_CAST(p.Result_nbr AS DOUBLE) AS admission_potassium,
      p.RequestDate as potassium_date,
      ROW_NUMBER() OVER (
        PARTITION BY np.ENCNTR_ID 
        ORDER BY p.RequestDate
      ) AS rn
    FROM nail_procedures np
    INNER JOIN 4_prod.rde.rde_pathology p 
      ON np.ENCNTR_ID = p.ENCNTR_ID
    WHERE p.Testdesc = 'Potassium Serum'
      AND p.Result_nbr IS NOT NULL
  ) ranked
  WHERE rn = 1
),

-- Get post-op Hemoglobin (2 weeks after admission test)
postop_hb AS (
  SELECT 
    np.ENCNTR_ID,
    CASE 
      WHEN p.resultUnit = 'g/L' THEN TRY_CAST(p.Result_nbr AS DOUBLE) / 10
      ELSE TRY_CAST(p.Result_nbr AS DOUBLE)
    END AS postop_hb,
    p.RequestDate as postop_hb_date
  FROM nail_procedures np
  INNER JOIN admission_hb ahb ON np.ENCNTR_ID = ahb.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE (p.Testdesc = 'Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin Concentration')
    AND (p.resultUnit = 'Grams/decilitre' OR p.resultUnit = 'g/L')
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > ahb.hb_date
    AND p.RequestDate <= DATE_ADD(ahb.hb_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get post-op eGFR (2 weeks after admission test)
postop_egfr AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS postop_egfr,
    p.RequestDate as postop_egfr_date
  FROM nail_procedures np
  INNER JOIN admission_egfr aegfr ON np.ENCNTR_ID = aegfr.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'estimated GFR'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > aegfr.egfr_date
    AND p.RequestDate <= DATE_ADD(aegfr.egfr_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get post-op Creatinine (2 weeks after admission test)
postop_creat AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS postop_creatinine,
    p.RequestDate as postop_creat_date
  FROM nail_procedures np
  INNER JOIN admission_creat acreat ON np.ENCNTR_ID = acreat.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Creatinine Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > acreat.creat_date
    AND p.RequestDate <= DATE_ADD(acreat.creat_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get post-op Urea (2 weeks after admission test)
postop_urea AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS postop_urea,
    p.RequestDate as postop_urea_date
  FROM nail_procedures np
  INNER JOIN admission_urea au ON np.ENCNTR_ID = au.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testcode = 'UREA'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > au.urea_date
    AND p.RequestDate <= DATE_ADD(au.urea_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get post-op Sodium (2 weeks after admission test)
postop_sodium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS postop_sodium,
    p.RequestDate as postop_sodium_date
  FROM nail_procedures np
  INNER JOIN admission_sodium asod ON np.ENCNTR_ID = asod.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Sodium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > asod.sodium_date
    AND p.RequestDate <= DATE_ADD(asod.sodium_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get post-op Potassium (2 weeks after admission test)
postop_potassium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS postop_potassium,
    p.RequestDate as postop_potassium_date
  FROM nail_procedures np
  INNER JOIN admission_potassium apot ON np.ENCNTR_ID = apot.ENCNTR_ID
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Potassium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate > apot.potassium_date
    AND p.RequestDate <= DATE_ADD(apot.potassium_date, 14)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY np.ENCNTR_ID ORDER BY p.RequestDate) = 1
),

-- Get 3-month follow-up blood tests (around 90 days after surgery, +/- 30 days)
threemonth_hb AS (
  SELECT 
    np.ENCNTR_ID,
    CASE 
      WHEN p.resultUnit = 'g/L' THEN TRY_CAST(p.Result_nbr AS DOUBLE) / 10
      ELSE TRY_CAST(p.Result_nbr AS DOUBLE)
    END AS threemonth_hb,
    p.RequestDate as threemonth_hb_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE (p.Testdesc = 'Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin Concentration')
    AND (p.resultUnit = 'Grams/decilitre' OR p.resultUnit = 'g/L')
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

threemonth_egfr AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS threemonth_egfr,
    p.RequestDate as threemonth_egfr_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'estimated GFR'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

threemonth_creat AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS threemonth_creatinine,
    p.RequestDate as threemonth_creat_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Creatinine Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

threemonth_urea AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS threemonth_urea,
    p.RequestDate as threemonth_urea_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testcode = 'UREA'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

threemonth_sodium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS threemonth_sodium,
    p.RequestDate as threemonth_sodium_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Sodium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

threemonth_potassium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS threemonth_potassium,
    p.RequestDate as threemonth_potassium_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Potassium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 60)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 120)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 90)))
  ) = 1
),

-- Get 12-month follow-up blood tests (around 365 days after surgery, +/- 60 days)
twelvemonth_hb AS (
  SELECT 
    np.ENCNTR_ID,
    CASE 
      WHEN p.resultUnit = 'g/L' THEN TRY_CAST(p.Result_nbr AS DOUBLE) / 10
      ELSE TRY_CAST(p.Result_nbr AS DOUBLE)
    END AS twelvemonth_hb,
    p.RequestDate as twelvemonth_hb_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE (p.Testdesc = 'Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin' 
      OR p.Testdesc = 'Mean Cell Haemoglobin Concentration')
    AND (p.resultUnit = 'Grams/decilitre' OR p.resultUnit = 'g/L')
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

twelvemonth_egfr AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS twelvemonth_egfr,
    p.RequestDate as twelvemonth_egfr_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'estimated GFR'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

twelvemonth_creat AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS twelvemonth_creatinine,
    p.RequestDate as twelvemonth_creat_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Creatinine Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

twelvemonth_urea AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS twelvemonth_urea,
    p.RequestDate as twelvemonth_urea_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testcode = 'UREA'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

twelvemonth_sodium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS twelvemonth_sodium,
    p.RequestDate as twelvemonth_sodium_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Sodium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

twelvemonth_potassium AS (
  SELECT 
    np.ENCNTR_ID,
    TRY_CAST(p.Result_nbr AS DOUBLE) AS twelvemonth_potassium,
    p.RequestDate as twelvemonth_potassium_date
  FROM nail_procedures np
  INNER JOIN 4_prod.rde.rde_pathology p ON np.PERSON_ID = p.PERSON_ID
  WHERE p.Testdesc = 'Potassium Serum'
    AND p.Result_nbr IS NOT NULL
    AND p.RequestDate >= DATE_ADD(np.surgery_date, 305)
    AND p.RequestDate <= DATE_ADD(np.surgery_date, 425)
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY np.ENCNTR_ID 
    ORDER BY ABS(TIMESTAMPDIFF(DAY, p.RequestDate, DATE_ADD(np.surgery_date, 365)))
  ) = 1
),

-- Get ethnicity (most recent record per person)
ethnicity_data AS (
  SELECT 
    PERSON_ID,
    Ethnicity
  FROM (
    SELECT 
      PERSON_ID,
      Ethnicity,
      ROW_NUMBER() OVER (PARTITION BY PERSON_ID ORDER BY MRN DESC) AS rn
    FROM 4_prod.rde.rde_patient_demographics
    WHERE Ethnicity IS NOT NULL
  ) ranked
  WHERE rn = 1
)

-- Main query combining all data
SELECT DISTINCT
  -- Patient Demographics
  pd.MRN AS Patient_Number,
  CASE 
    WHEN TIMESTAMPDIFF(YEAR, pd.Date_of_Birth, np.surgery_date) >= 89 THEN '+89'
    ELSE CAST(TIMESTAMPDIFF(YEAR, pd.Date_of_Birth, np.surgery_date) AS VARCHAR(10))
  END AS Age,
  pd.Gender AS Sex,
  ed.Ethnicity,
  bmi.height_cm AS Height_cm,
  bmi.weight_kg AS Weight_kg,
  bmi.BMI,
  COALESCE(bmi.is_obese, 'Unknown') AS Obesity_BMI_30_plus,
  
  -- Comorbidities
  COALESCE(d.has_diabetes, 'N') AS Diabetes,
  COALESCE(s.is_smoker, 'N') AS Tobacco_Use,
  
  -- Surgical Details
  YEAR(np.surgery_date) AS Year_of_Surgery,
  YEAR(np.surgery_date) AS Year_of_OTS,
  CONCAT('Q', QUARTER(np.surgery_date)) AS Quarter_of_OTS,
  np.surgery_date AS Surgery_Date,
  COALESCE(li.laterality, np.SIDE_OF_PROCEDURE, 'Unknown') AS Laterality,
  
  -- Nail Type
  CASE WHEN np.is_gentamicin_coated = 'Y' THEN 'Y' ELSE 'N' END AS GCN_ETN_PROtect,
  CASE WHEN np.is_gentamicin_coated = 'N' THEN 'Y' ELSE 'N' END AS NGCN_ETN,
  
  -- Admission Laboratory Values
  ahb.admission_hb AS Admission_Hb,
  aegfr.admission_egfr AS Admission_eGFR,
  au.admission_urea AS Admission_Urea,
  asod.admission_sodium AS Admission_Na,
  apot.admission_potassium AS Admission_K,
  acreat.admission_creatinine AS Admission_Creatinine,
  
  -- Post-operative Laboratory Values (within 2 weeks)
  pohb.postop_hb AS PostOp_Hb,
  poegfr.postop_egfr AS PostOp_eGFR,
  pou.postop_urea AS PostOp_Urea,
  posod.postop_sodium AS PostOp_Na,
  popot.postop_potassium AS PostOp_K,
  pocreat.postop_creatinine AS PostOp_Creatinine,
  
  -- 3-Month Follow-up Laboratory Values
  tmhb.threemonth_hb AS ThreeMonth_Hb,
  tmegfr.threemonth_egfr AS ThreeMonth_eGFR,
  tmu.threemonth_urea AS ThreeMonth_Urea,
  tmsod.threemonth_sodium AS ThreeMonth_Na,
  tmpot.threemonth_potassium AS ThreeMonth_K,
  tmcreat.threemonth_creatinine AS ThreeMonth_Creatinine,
  
  -- 12-Month Follow-up Laboratory Values
  twmhb.twelvemonth_hb AS TwelveMonth_Hb,
  twmegfr.twelvemonth_egfr AS TwelveMonth_eGFR,
  twmu.twelvemonth_urea AS TwelveMonth_Urea,
  twmsod.twelvemonth_sodium AS TwelveMonth_Na,
  twmpot.twelvemonth_potassium AS TwelveMonth_K,
  twmcreat.twelvemonth_creatinine AS TwelveMonth_Creatinine,
  
  -- Additional Information
  np.ENCNTR_ID,
  pd.PERSON_ID,
  pc.procedure_list AS Procedure_Codes,
  dc.diagnosis_list AS Diagnosis_Codes,
  np.IMPLANT_DESCRIPTION AS Implant_Details,
  np.MANUFACTURER

FROM nail_procedures np
INNER JOIN 4_prod.rde.rde_patient_demographics pd 
  ON np.PERSON_ID = pd.PERSON_ID
LEFT JOIN laterality_info li 
  ON np.ENCNTR_ID = li.ENCNTR_ID
LEFT JOIN procedure_codes pc 
  ON np.ENCNTR_ID = pc.ENCNTR_ID
LEFT JOIN diagnosis_codes dc 
  ON np.ENCNTR_ID = dc.ENCNTR_ID
LEFT JOIN diabetes_check d 
  ON np.PERSON_ID = d.person_id
LEFT JOIN smoker_check s 
  ON np.PERSON_ID = s.person_id
LEFT JOIN bmi_calc bmi 
  ON np.PERSON_ID = bmi.PERSON_ID
LEFT JOIN ethnicity_data ed
  ON np.PERSON_ID = ed.PERSON_ID
  
-- Admission blood tests
LEFT JOIN admission_hb ahb 
  ON np.ENCNTR_ID = ahb.ENCNTR_ID
LEFT JOIN admission_egfr aegfr 
  ON np.ENCNTR_ID = aegfr.ENCNTR_ID
LEFT JOIN admission_creat acreat 
  ON np.ENCNTR_ID = acreat.ENCNTR_ID
LEFT JOIN admission_urea au 
  ON np.ENCNTR_ID = au.ENCNTR_ID
LEFT JOIN admission_sodium asod 
  ON np.ENCNTR_ID = asod.ENCNTR_ID
LEFT JOIN admission_potassium apot 
  ON np.ENCNTR_ID = apot.ENCNTR_ID
  
-- Post-operative blood tests
LEFT JOIN postop_hb pohb
  ON np.ENCNTR_ID = pohb.ENCNTR_ID
LEFT JOIN postop_egfr poegfr
  ON np.ENCNTR_ID = poegfr.ENCNTR_ID
LEFT JOIN postop_creat pocreat
  ON np.ENCNTR_ID = pocreat.ENCNTR_ID
LEFT JOIN postop_urea pou
  ON np.ENCNTR_ID = pou.ENCNTR_ID
LEFT JOIN postop_sodium posod
  ON np.ENCNTR_ID = posod.ENCNTR_ID
LEFT JOIN postop_potassium popot
  ON np.ENCNTR_ID = popot.ENCNTR_ID
  
-- 3-month follow-up blood tests
LEFT JOIN threemonth_hb tmhb
  ON np.ENCNTR_ID = tmhb.ENCNTR_ID
LEFT JOIN threemonth_egfr tmegfr
  ON np.ENCNTR_ID = tmegfr.ENCNTR_ID
LEFT JOIN threemonth_creat tmcreat
  ON np.ENCNTR_ID = tmcreat.ENCNTR_ID
LEFT JOIN threemonth_urea tmu
  ON np.ENCNTR_ID = tmu.ENCNTR_ID
LEFT JOIN threemonth_sodium tmsod
  ON np.ENCNTR_ID = tmsod.ENCNTR_ID
LEFT JOIN threemonth_potassium tmpot
  ON np.ENCNTR_ID = tmpot.ENCNTR_ID
  
-- 12-month follow-up blood tests
LEFT JOIN twelvemonth_hb twmhb
  ON np.ENCNTR_ID = twmhb.ENCNTR_ID
LEFT JOIN twelvemonth_egfr twmegfr
  ON np.ENCNTR_ID = twmegfr.ENCNTR_ID
LEFT JOIN twelvemonth_creat twmcreat
  ON np.ENCNTR_ID = twmcreat.ENCNTR_ID
LEFT JOIN twelvemonth_urea twmu
  ON np.ENCNTR_ID = twmu.ENCNTR_ID
LEFT JOIN twelvemonth_sodium twmsod
  ON np.ENCNTR_ID = twmsod.ENCNTR_ID
LEFT JOIN twelvemonth_potassium twmpot
  ON np.ENCNTR_ID = twmpot.ENCNTR_ID